# Extracting N2 3' UTR regions from other genomes

# Blasting N2 3' UTR regions to CB4856

### 1) download the CB4856 genome from WormBase FTP site:  
ftp://ftp.wormbase.org/pub/wormbase/species/c_elegans/PRJNA275000/sequence/genomic/

I believe the CB4856 assembly is PRJNA275000. I chose the WS268 unmasked assembly, as we think WS268 matches what is used in CENDR:
`c_elegans.PRJNA275000.WS268.genomic.fa.gz`

### 2) convert the genome sequences into a local blast database. Requires BLAST+ to be installed

In [10]:
gunzip ../Genome_seqs/c_elegans.PRJNA275000.WS268.genomic.fa.gz
makeblastdb -in ../Genome_seqs/c_elegans.PRJNA275000.WS268.genomic.fa -out "../Genome_seqs/CB4856_WS268_genomic" \
-taxid 6239 -dbtype nucl -parse_seqids

gzip: ../Genome_seqs/c_elegans.PRJNA275000.WS268.genomic.fa.gz: No such file or directory


Building a new DB, current time: 05/14/2019 14:37:12
New DB name:   /home/ksilliman/Projects/PD_RNAworms/Genome_seqs/CB4856_WS268_genomic
New DB title:  ../Genome_seqs/c_elegans.PRJNA275000.WS268.genomic.fa
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 7 sequences in 0.840637 seconds.


### 3) Download the 3'UTR sequencefrom ParaSite through WomBase, as described on the [WormBase FAQ page](https://www.wormbase.org/about/Frequently_asked_questions).

In [16]:
# Set the variable name for the ParaSite download and the filtered fasta file
# Change these anytime you download a new file 
UTR = "martquery_0506213417_153.txt"
UTRf = "UTR3_3417_WS269_filt.fa"

In [18]:
%expand
# Unzip 3'UTR download
gunzip {UTR}.gz

### 4) For some reason, the download includes a lot of sequences that say "Sequence unavailable". These need to be removed.

In [19]:
%expand
# Remove all sequences that say "Sequence unavailable". Print the total number of sequences.
IN = open("{UTR}","r")
OUT = open("{UTRf}","w")

n = 0
seq = ""
for line in IN:
    if ">" in line:
        if seq != "":
            OUT.write(header+seq+"\n")
            n += 1
            seq = ""
        header = line
    else:
        if "unavailable" not in line:
            seq = seq + line.strip()
        else:
            seq = ""
IN.close()
OUT.close()
print(n)

25978


In [20]:
%expand
head {UTRf}

>WBGene00003525|17486871|17486981|4R79.1a.1
TCTTCTATCTGGTGTTATTTATTTTGTTGCTTATTGTTCCATGACGTGTGTATAATGTAATTCTGAAAGCCAATTTTTTCATTTTTTGAAAATATTTATATAATTTATACT
>WBGene00004098|10383977|10384063|AC3.4.1
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTCTCATTTTTTTCACAATAAATAATAATAATGCTTG
>WBGene00007071|10393357|10393504|AC3.5a.2
ATGAATTTCCATACAATGACAAAAACTATTAGTGACAGATAACATAAACACTTGATTTTATTTATTAATGTGAAACCGGTCAGAGTTCATAATTTTTGTTGTAACTTGTGTTTGCCTCAACATTGAATAAAATGTTTATAAATCGGAC
>WBGene00007072|10397735|10397817|AC3.7.1
TTTTAAAAAGTTTTATTTGCTATCAATTTGTATCTCTTGTTGATTTAATTCATATTTGAGCCTTAATAAACTGTCTAATCTGC
>WBGene00000024|10380340|10380425|AC3.3.1
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTTTTTCTTTTTTTTCACAATAACATTGCTTGAAAT


### 5) Blast N2 3' UTR sequences against the CB4856 genome.
Requires an e-value of < 1e-3, at least 80% coverage of the query sequence, and only the top 3 hits.

In [34]:
%expand
blastn -query {UTRf} -task megablast -outfmt 10 -word_size 11 \
-db ../Genome_seqs/CB4856_WS268_genomic -out 3UTR_blast.out \
-num_threads 2 -max_hsps 3 -evalue 1e-3 -qcov_hsp_perc 80

In [1]:
# Look at output of blast
# Columns are: Fasta header,
# aligned chromosome,
# # of identical matches,
# alignment length,
# # of mismatches,
# # of gaps,
# alignment start pos query,
# alignment end pos query,
# alignment start pos in target
# alignment end pos in target
# evalue
# bit score
head -n 20 3UTR_blast.out

WBGene00003525|17486871|17486981|4R79.1a.1,IV,100.00,111,0,0,1,111,17178578,17178468,8e-53,206
WBGene00004098|10383977|10384063|AC3.4.1,V,100.00,87,0,0,1,87,10068540,10068626,1e-39,161
WBGene00004098|10383977|10384063|AC3.4.1,V,91.78,73,4,1,1,73,10064988,10064918,3e-21,100
WBGene00007071|10393357|10393504|AC3.5a.2,V,100.00,148,0,0,1,148,10077922,10078069,3e-73,274
WBGene00007072|10397735|10397817|AC3.7.1,V,100.00,83,0,0,1,83,10082300,10082382,2e-37,154
WBGene00000024|10380340|10380425|AC3.3.1,V,94.52,73,2,1,1,73,10068540,10068610,1e-24,111
WBGene00004964|10385685|10385711|AC3.10.1,V,100.00,27,0,0,1,27,10070248,10070274,3e-07,51.0
WBGene00007063|4664|4717|2L52.1a.1,II,100.00,54,0,0,1,54,4663,4716,1e-21,100
WBGene00007073|10401223|10401307|AC3.8.1,V,98.82,85,1,0,1,85,10085789,10085873,7e-37,152
WBGene00007070|10374208|10374245|AC3.2.1,V,100.00,38,0,0,1,38,10058907,10058944,6e-13,71.3
WBGene00007071|10393357|10393504|AC3.5a.1,V,100.00,148,0,0,1,148,10077922,10078069,3e-73,274
WBGene000070

Some N2 3'UTR loci are aligniing with high confidence to multiple regions of the CB4856 genome. For example, transcript AC3.4.1 aligns to two locations, one alignment with 100% match (evalue 1e-39) and one alignment with 4 mismatches and one gap (evalue 3e-21).  
Another issue: sometimes the retrieved CB4856 sequence is shorter than the N2 3'UTR sequence. E.g., A3.3.1 retrieves a 73nt long sequence from CB4856, but the N2 query sequence is 87 nt. See below. Unsure how to fix this or if it needs fixing.

In [24]:
# how to extract the sequence from 
blastdbcmd -db ../Genome_seqs/CB4856_WS268_genomic -entry 'V' -range 10068540-10068610
blastdbcmd -db ../Genome_seqs/CB4856_WS268_genomic -entry 'V' -range 10068540-10068610 | tail -n 1 | wc -c

>lcl|V:10068540-10068610 length=20182852
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTCTCATTTTTTTCACAATAA
72


In [40]:
blastdbcmd -db ../Genome_seqs/CB4856_WS268_genomic -entry 'V' -range 10068540-10068625

>lcl|V:10068540-10068625 length=20182852
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTCTCATTTTTTTCACAATAAATAATAATA
ATGCTT


# 
blastdbcmd -help

 -entry_batch <File_In>
   Input file for batch processing (Format: one entry per line, seq id 
   followed by optional space-delimited specifier(s)
   [range|strand|mask_algo_id]

In [25]:
%expand
grep "WBGene00000024|10380340|10380425|AC3.3.1" {UTRf} -A 1 
grep "WBGene00000024|10380340|10380425|AC3.3.1" {UTRf} -A 1 | tail -n 1 | wc -c

>WBGene00000024|10380340|10380425|AC3.3.1
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTTTTTCTTTTTTTTCACAATAACATTGCTTGAAAT
87


In [39]:
%expand
blastn -query {UTRf} -task megablast -word_size 11 \
-db ../Genome_seqs/CB4856_WS268_genomic -out 3UTR_blastalign.out \
-num_threads 2 -max_hsps 3 -evalue 1e-3 -qcov_hsp_perc 80

In [ ]:
# make input file for parsing seqs


# Using CENDR data

In [1]:
gunzip ../Genome_seqs/c_elegans.PRJNA13758.WS269.annotations.gff3.gz

In [2]:
awk 'BEGIN {FS="\t";OFS="\t"} {if($9 ~ "CB4856") print}' ../Genome_seqs/c_elegans.PRJNA13758.WS269.annotations.gff3 > WS269_Hawaiian_GFF3_lines.txt

In [3]:
head WS269_Hawaiian_GFF3_lines.txt

I	Variation_project_Polymorphism	SNP	1222	1222	.	+	.	variation=WBVar00146751;public_name=WBVar00146751;other_name=haw1,cewivar00160085;strain=CB4856;polymorphism=1;substitution=A/C
I	Variation_project_Polymorphism	complex_substitution	1369	1388	.	+	.	variation=WBVar02088481;public_name=WBVar02088481;other_name=cewivar00818301;strain=CB4856;polymorphism=1;insertion=CTTATTATTGATGTCAAAGAAGC
I	Variation_project_Polymorphism	SNP	1713	1713	.	+	.	variation=WBVar01273539;public_name=WBVar01273539;other_name=cewivar00160086;strain=CB4856;polymorphism=1;substitution=T/G
I	Variation_project_Polymorphism	SNP	1933	1933	.	+	.	variation=WBVar01529474;public_name=WBVar01529474;other_name=cewivar00065822;strain=AB3,CB4853,CB4854,CB4856,ED3021,ED3040,ED3042,ED3049,JU263,JU300,JU312,JU322,JU345,JU361,JU397,JU775,JU1088,JU1171,JU1401,JU1652,KR314,LKC34,MY2,MY14,MY16,PX174;polymorphism=1;substitution=C/G
I	Variation_project_Polymorphism	insertion_site	1943	1943	.	+	.	variation=WBVar01955456;public_name=WBV

In [4]:
head UTR3_3417_WS269_filt.fa

>WBGene00003525|17486871|17486981|4R79.1a.1
TCTTCTATCTGGTGTTATTTATTTTGTTGCTTATTGTTCCATGACGTGTGTATAATGTAATTCTGAAAGCCAATTTTTTCATTTTTTGAAAATATTTATATAATTTATACT
>WBGene00004098|10383977|10384063|AC3.4.1
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTCTCATTTTTTTCACAATAAATAATAATAATGCTTG
>WBGene00007071|10393357|10393504|AC3.5a.2
ATGAATTTCCATACAATGACAAAAACTATTAGTGACAGATAACATAAACACTTGATTTTATTTATTAATGTGAAACCGGTCAGAGTTCATAATTTTTGTTGTAACTTGTGTTTGCCTCAACATTGAATAAAATGTTTATAAATCGGAC
>WBGene00007072|10397735|10397817|AC3.7.1
TTTTAAAAAGTTTTATTTGCTATCAATTTGTATCTCTTGTTGATTTAATTCATATTTGAGCCTTAATAAACTGTCTAATCTGC
>WBGene00000024|10380340|10380425|AC3.3.1
ACATCGAATGCGTAACTTTGACATCAGTTCTCTGTATATATGACACAATTTTTTTTCTTTTTTTTCACAATAACATTGCTTGAAAT


In [26]:
gunzip ../Genome_seqs/c_elegans.PRJNA275000.WS268.annotations.gff3.gz

In [41]:
grep "AC3.3" ../Genome_seqs/c_elegans.PRJNA275000.WS268.annotations.gff3

V	celegans_proteins-BLASTX	protein_match	10064992	10066215	2303	-	.	ID=wormpepx.5024800;Name=AC3.3;Target=AC3.3 10064992 10066215;Gap=M1224 
V	celegans_proteins-BLASTX	protein_match	10066262	10066348	78	-	.	ID=wormpepx.5024666;Name=AC3.3;Target=AC3.3 10066262 10066348;Gap=M87 
V	celegans_proteins-BLASTX	protein_match	4879688	4879795	44	+	.	ID=wormpepx.5073131;Name=AC3.3;Target=AC3.3 4879688 4879795;Gap=M108 
V	celegans_proteins-BLASTX	protein_match	8877326	8877379	40	+	.	ID=wormpepx.3259547;Name=AC3.3;Target=AC3.3 8877326 8877379;Gap=M54 
V	celegans_proteins-BLASTX	protein_match	10067313	10068536	2295	+	.	ID=wormpepx.5024569;Name=AC3.3;Target=AC3.3 10067313 10068536;Gap=M1224 
X	celegans_proteins-BLASTX	protein_match	3709067	3709129	40	-	.	ID=wormpepx.4578565;Name=AC3.3;Target=AC3.3 3709067 3709129;Gap=M63 
IV	celegans_proteins-BLASTX	protein_match	15060362	15061561	1944	-	.	ID=wormpepx.2337780;Name=AC3.3;Target=AC3.3 15060362 15061561;Gap=M81 I6 M222 D3 M201 I6 M429 I9 M264 
IV	celega

In [34]:
gunzip ../Genome_seqs/File_S1_Celegans_N2_to_CB4856_lastz_alignment.txt.gz

In [38]:
head -n 20 ../Genome_seqs/File_S1_Celegans_N2_to_CB4856_lastz_alignment.txt

#Thompson et al.
#File S1 LastZ alignments between C. elegans N2 (WS230) and CB4856
# If the coordinate is not found in this file, then that region was
# not aligned by lastZ.
#Column 1: Chromosome
#Column 2: C. elegans N2 WS230 base
#Column 3: C. elegans N2 WS230 coordinate
#Column 4: C. elegans CB4856 base
#Column 5: C. elegans CB4856 coordinate

CeI G 1 G 1
CeI C 2 C 2
CeI C 3 C 3
CeI T 4 T 4
CeI A 5 A 5
CeI A 6 A 6
CeI G 7 G 7
CeI C 8 C 8
CeI C 9 C 9
CeI T 10 T 10
